# **EXPLORATORY DATA ANALYSIS FOR M5**

## **INITIALIZATION**

In [1]:
# load required packages
import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import pylab as pl

import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

from scipy import stats
from scipy.stats import norm, skew

import gc
import lightgbm as lgb

In [2]:
# ignore warnings from sklearn and seaborn
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# pandas output format
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
pd.options.display.max_columns = 50

In [3]:
# check files available
from subprocess import check_output
print(check_output(['ls', os.getcwd()]).decode('utf8'))

calendar.csv
M5-Competitors-Guide-Final-10-March-2020.odt
m5-forecasting-eda.ipynb
sales_train_validation.csv
sample_submission.csv
sell_prices.csv
SGB-m5-forecasting.ipynb



## **EXPLORATION**

In [20]:
cal_dtypes = {'event_name_1': 'category', 'event_name_2': 'category', 
              'event_type_1': 'category', 'event_type_2': 'category',
              'weekday': 'category', 'wm_yr_wk': 'int16', 'wday': 'int16',
              'month': 'int16', 'year': 'int16', 'snap_CA': 'float32', 
              'snap_TX': 'float32', 'snap_WI': 'float32'}
price_dtypes = {'store_id': 'category', 'item_id': 'category', 'wm_yr_wk': 'int16',
               'sell_price': 'float32'}

In [9]:
# parameters for constructing time series
h = 28 # forecast horizon
max_lags = 57
tr_last = 1913 # last training observation
fday = datetime(2016, 4, 25) # forecast start date
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [18]:
# construct time series
def create_df(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv('sell_prices.csv', dtype = price_dtypes)
    for col, col_dtype in price_dtypes.items():
        if col_dtype == 'category':
            prices[col] = prices[col].cat.codes.astype('int16')
            prices[col] -= prices[col].min() # scaling
    cal = pd.read_csv('calendar.csv', dtype = cal_dtypes)
    cal['date'] = pd.to_datetime(cal['date'])
    for col, col_dtype in cal_dtypes.items():
        if col_dtype == 'category':
            cal[col] = cal[col].cat.codes.astype('int16')
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train else tr_last - max_lags, first_day)
    numcols = [f'd_{day}' for day in range(start_day, tr_last+1)] #sales data rolling window
    catcols = ['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id']
    dtype = {numcol: 'float32' for numcol in numcols}
    dtype.update({col: 'category' for col in catcols if col != 'id'})
    df = pd.read_csv('sales_train_validation.csv', nrows = nrows, 
                     usecols = catcols + numcols, dtype = dtype)
    for col in catcols:
        if col != 'id':
            df[col] = df[col].cat.codes.astype('int16')
            df[col] -= df[col].min()
    if not is_train:
        for day in range(tr_last + 1, tr_last + 28 + 1):
            df[f'd_{day}'] = np.nan
    df = pd.melt(df, 
                 id_vars = catcols,
                 value_vars = [col for col in df.columns if col.startswith('d_')], # numeric
                 var_name = 'd', # day
                 value_name = 'sales')
    df = df.merge(cal, on='d', copy = False)
    df = df.merge(prices, on = ['store_id', 'item_id', 'wm_yr_wk'], copy=False)
    return df 

In [14]:
# create forecast series
def create_fea(df):
    lags = [7, 28]
    lag_cols = [f'lag_{lag}' for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[['id', 'sales']].groupby('id')['sales'].shift(lag)
        
    wins = [7, 28] # windows
    for win in wins:
        for lag, lag_col in zip(lags, lag_cols):
            df[f'rmean_{lag}_{win}'] = df[['id', lag_col]].groupby('id')[lag_col].transform(lambda x: x.rolling(win).mean())
    
    date_features = {
        'wday': 'weekday',
        'week': 'weekofyear',
        'month': 'month',
        'quarter': 'quarter',
        'year': 'year',
        'mday': 'day'}
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in df.columns:
            df[date_feat_name] = df[date_feat_name].astype('int16')
        else:
            df[date_feat_name] = getattr(df['date'].dt, date_feat_func).astype('int16')

In [21]:
%%time
df = create_df(is_train=True, first_day = 50500) #skip days to save on memory
df.shape

MemoryError: Unable to allocate array with shape (40718219, 1) and data type float32

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
%%

In [ ]:
# load sale date and sale price features
sale_date = pd.read_csv('calendar.csv') # date of sales
sale_price = pd.read_csv('sell_prices.csv') # price of items sold
labels = sale_date['wm_yr_wk'].values
#print(calendar.info())
print(sale_date.head())

In [ ]:
# clean up sale price table
sale_price['id'] =  sale_price['item_id'] + '_' + sale_price['store_id'] + '_validation'
sale_price.drop(['store_id', 'item_id'], axis=1, inplace=True)

In [ ]:
# clean up df, group by id and transpose
df2 = df.copy()
ls = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df2.drop(ls, axis=1, inplace=True)
df2 = df2.set_index('id').T
print(df2.head())

In [ ]:
# test
tmp = pd.DataFrame(df2.iloc[:, 1])
tmp['wm_yr_wk'] = labels[:tmp.shape[0]]
tmp2 = sale_price[sale_price['id'] == 'HOBBIES_1_002_CA_1_validation']
new_df = pd.merge(tmp, tmp2, on = 'wm_yr_wk', how = 'left')
print(new_df.tail())

In [ ]:
# split data
sales = dict()
for j in range(1, df2.shape[1]):
    sales_id = df2.columns[j]
    tmp = pd.DataFrame(df2.iloc[:, j])
    tmp['wm_yr_wk'] = labels[:tmp.shape[0]]
    tmp2 = sale_price[sale_price['id'] == sales_id]
    tmp2 = pd.merge(tmp, tmp2, on = 'wm_yr_wk', how = 'left')
    sales[sales_id] = tmp2.drop('id', axis = 1)

In [ ]:
# start off with 1 item


In [ ]:
# split day columns into 20 chunks for train and test
#print(df.columns[-1])
chunks = dict()
for chunk_id in range(20):
    col_start = 90 * chunk_id + 1
    chunks[chunk_id] = df.iloc[:, col_start:col_start + 90]
chunks[20] = df.iloc[:, 1801:1914]

In [ ]:
# define function to split each chunk into train and test
def split_train_test(chunks):
    train, test = list(), list()
    # first 60 days of observations for train
    cut_point = 60
    # enumerate chunks
    for rows, cols in chunks.items():
        # split chunk columns by 'position_within_chunk'
        train_cols = cols.iloc[:, :cut_point]
        test_cols = cols.iloc[:, cut_point:]
        if len(train_cols) == 0 or len(test_cols) == 0:
            continue
        train.append(train_cols)
        test.append(test_cols)
    return train, test

In [ ]:
# return a list of relative forecast lead times
# in reality need to forecast day+1, day+2, ... day+28
#def get_lead_times():
#    return [1, 7, 14, 28]

In [ ]:
# split data into train and test
train, test = split_train_test(chunks)

In [ ]:
# convert the test data set in chunks to [chunk][variable][time] format
def prepare_test_forecasts(test_chunks):
    predictions = list()
    # enumerate chunks to forecast
    for cols in test_chunks:
        # enumerate targets for chunk
        chunk_predictions = list()
        for j in range(cols.shape[0]):
            yhat = cols[j, :]
            chunk_predictions.append(yhat)
        chunk_predictions = np.array(chunk_predictions)
        predictions.append(chunk_predictions)

In [ ]:
# distribution plot of target variable
y = train['SalePrice']
print(y.describe())

sns.distplot(y, fit=norm);
(mu, sigma) = norm.fit(y)
print('\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f})'.format(mu, sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# QQ plot
fig = plt.figure()
res = stats.probplot(y, plot=plt)
plt.show()

# therefore, logscale distribution is better
#plt.figure(figsize=(10,10))
#plt.hist(y, bins=np.geomspace(y.min(), y.max(), 50))
#plt.xscale('log')
#plt.show()

## **FEATURE ENGINEERING**

In [ ]:
# apply log transform to target variable
train['SalePrice'] = np.log1p(train['SalePrice'])
sns.distplot(train['SalePrice'], fit=norm); # check

In [ ]:
# data manipulation
n_train = train.shape[0]; n_test = test.shape[0]
y = train['SalePrice'].values
df = pd.concat((train, test)).reset_index(drop=True)
del df['SalePrice']
print(n_train)

In [ ]:
# deal with missing data
df_nan = df.isnull().sum() / len(df) * 100
df_nan = df_nan.drop(df_nan[df_nan == 0].index).sort_values(ascending=False)
print(df_nan[:10])

f, ax = plt.subplots(figsize=(10,10))
plt.xticks(rotation='90')
sns.barplot(x=df_nan.index[:10], y=df_nan[:10])
plt.xlabel('Features', fontsize=12)
plt.ylabel('% missing', fontsize=12)
plt.title('% missing by feature', fontsize=12)

In [ ]:
# x-correlation map
corrmat = train.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corrmat, vmax=0.9)
plt.tight_layout()

In [ ]:
# deal with missing and error values
df2 = df.copy()
df2.replace(r'^\s*$', np.nan, regex=True)

# all below from https://www.kaggle.com/juliensiems/cleaning-new-features-gps-coordinates-included

df2['LotFrontage'] = df2.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# replace missing values with zeros
ls = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtUnfSF', 'TotalBsmtSF',
      'EnclosedPorch', 'Fireplaces', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'KitchenAbvGr', 
      'MasVnrArea', 'MiscVal', 'OpenPorchSF', 'PoolArea','ScreenPorch', 'TotRmsAbvGrd', 'WoodDeckSF']
for f in ls:
    df2[f].fillna(0, inplace=True)
    
# relace missing values with labels
ls_no = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Fence',
        'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'MiscFeature','PoolQC']
for f in ls_no:
    df2[f].fillna("No", inplace=True)
    
# replace missing values with other labels
ls_ta = ['ExterCond', 'ExterQual', 'HeatingQC', 'KitchenQual']
ls_norm = ['Condition1', 'Condition2']
for f in ls_ta:
    df2[f].fillna("TA", inplace=True)
for f in ls_norm:
    df2[f].fillna("Norm", inplace=True)

df2['Alley'].fillna('None', inplace=True)
df2['CentralAir'].fillna('N', inplace=True)
df2['PavedDrive'].fillna('N', inplace=True)
df2['MasVnrType'].fillna('None', inplace=True)

ls = ['MSZoning', 'Utilities', 'Electrical', 'SaleCondition', 'SaleType', 'LotShape', 'Functional',
      'Exterior2nd', 'Exterior1st']
for f in ls:
    df2[f].fillna(df2[f].mode()[0], inplace=True)

# add features to replace neighborhood by its coordinates

df2['lat'] = df2['Neighborhood'].values
df2['lon'] = df2['Neighborhood'].values

df2['lat'].replace({'Blmngtn' : 42.062806,
                                               'Blueste' : 42.009408,
                                                'BrDale' : 42.052500,
                                                'BrkSide': 42.033590,
                                                'ClearCr': 42.025425,
                                                'CollgCr': 42.021051,
                                                'Crawfor': 42.025949,
                                                'Edwards': 42.022800,
                                                'Gilbert': 42.027885,
                                                'GrnHill': 42.000854,
                                                'IDOTRR' : 42.019208,
                                                'Landmrk': 42.044777,
                                                'MeadowV': 41.991866,
                                                'Mitchel': 42.031307,
                                                'NAmes'  : 42.042966,
                                                'NoRidge': 42.050307,
                                                'NPkVill': 42.050207,
                                                'NridgHt': 42.060356,
                                                'NWAmes' : 42.051321,
                                                'OldTown': 42.028863,
                                                'SWISU'  : 42.017578,
                                                'Sawyer' : 42.033611,
                                                'SawyerW': 42.035540,
                                                'Somerst': 42.052191,
                                                'StoneBr': 42.060752,
                                                'Timber' : 41.998132,
                                                'Veenker': 42.040106}, inplace=True)

df2['lon'].replace({'Blmngtn' : -93.639963,
                                               'Blueste' : -93.645543,
                                                'BrDale' : -93.628821,
                                                'BrkSide': -93.627552,
                                                'ClearCr': -93.675741,
                                                'CollgCr': -93.685643,
                                                'Crawfor': -93.620215,
                                                'Edwards': -93.663040,
                                                'Gilbert': -93.615692,
                                                'GrnHill': -93.643377,
                                                'IDOTRR' : -93.623401,
                                                'Landmrk': -93.646239,
                                                'MeadowV': -93.602441,
                                                'Mitchel': -93.626967,
                                                'NAmes'  : -93.613556,
                                                'NoRidge': -93.656045,
                                                'NPkVill': -93.625827,
                                                'NridgHt': -93.657107,
                                                'NWAmes' : -93.633798,
                                                'OldTown': -93.615497,
                                                'SWISU'  : -93.651283,
                                                'Sawyer' : -93.669348,
                                                'SawyerW': -93.685131,
                                                'Somerst': -93.643479,
                                                'StoneBr': -93.628955,
                                                'Timber' : -93.648335,
                                                'Veenker': -93.657032}, inplace=True)

# create new features by combining existing features
df2['IsRegularLotShape'] = (df2['LotShape'] =='Reg') * 1
df2['IsLandLevel'] = (df2['LandContour'] == 'Lvl') * 1
df2['IsLandSlopeGentle'] = (df2['LandSlope'] == 'Gtl') * 1
df2['IsElectricalSBrkr'] = (df2['Electrical'] == 'SBrkr') * 1
df2['IsGarageDetached'] = (df2['GarageType'] == 'Detchd') * 1
df2['IsPavedDrive'] = (df2['PavedDrive'] == 'Y') * 1
df2['HasShed'] = (df2['MiscFeature'] == 'Shed') * 1.
df2['Remodeled'] = (df2['YearRemodAdd'] != df2['YearBuilt']) * 1
df2['RecentRemodel'] = (df2['YearRemodAdd'] == df2['YrSold']) * 1
df2['VeryNewHouse'] = (df2['YearBuilt'] == df2['YrSold']) * 1
df2['HasMasVnr'] = (df2['MasVnrArea'] == 0) * 1
df2['HasWoodDeck'] = (df2['WoodDeckSF'] == 0) * 1
df2['HasOpenPorch'] = (df2['OpenPorchSF'] == 0) * 1
df2['HasEnclosedPorch'] = (df2['EnclosedPorch'] == 0) * 1
df2['Has3SsnPorch'] = (df2['3SsnPorch'] == 0) * 1
df2['HasScreenPorch'] = (df2['ScreenPorch'] == 0) * 1

# encode categorical variables
df2 = df2.replace({'Alley' : {'Grvl' : 1, 'Pave' : 2},
                           'BsmtCond' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5},
                           'BsmtExposure' : {'No' : 0, 'Mn' : 1, 'Av': 2, 'Gd' : 3},
                           'BsmtFinType1' : {'No' : 0, 'Unf' : 1, 'LwQ': 2, 'Rec' : 3, 'BLQ' : 4,
                                             'ALQ' : 5, 'GLQ' : 6},
                           'BsmtFinType2' : {'No' : 0, 'Unf' : 1, 'LwQ': 2, 'Rec' : 3, 'BLQ' : 4,
                                             'ALQ' : 5, 'GLQ' : 6},
                           'BsmtQual' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA': 3, 'Gd' : 4, 'Ex' : 5},
                           'ExterCond' : {'Po' : 1, 'Fa' : 2, 'TA': 3, 'Gd': 4, 'Ex' : 5},
                           'ExterQual' : {'Po' : 1, 'Fa' : 2, 'TA': 3, 'Gd': 4, 'Ex' : 5},
                           'FireplaceQu' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5},
                           'Functional' : {'Sal' : 1, 'Sev' : 2, 'Maj2' : 3, 'Maj1' : 4, 'Mod': 5,
                                           'Min2' : 6, 'Min1' : 7, 'Typ' : 8},
                           'GarageCond' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5},
                           'GarageQual' : {'No' : 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5},
                           'HeatingQC' : {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5},
                           'KitchenQual' : {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5},
                           'LandSlope' : {'Sev' : 1, 'Mod' : 2, 'Gtl' : 3},
                           'LotShape' : {'IR3' : 1, 'IR2' : 2, 'IR1' : 3, 'Reg' : 4},
                           'PavedDrive' : {'N' : 0, 'P' : 1, 'Y' : 2},
                           'PoolQC' : {'No' : 0, 'Fa' : 1, 'TA' : 2, 'Gd' : 3, 'Ex' : 4},
                           'Street' : {'Grvl' : 1, 'Pave' : 2},
                           'Utilities' : {'ELO' : 1, 'NoSeWa' : 2, 'NoSewr' : 3, 'AllPub' : 4}})

# combining existing features
df2['OverallGrade'] = df2['OverallQual'] * df2['OverallCond']
df2['GarageGrade'] = df2['GarageQual'] * df2['GarageCond']
df2['ExterGrade'] = df2['ExterQual'] * df2['ExterCond']
df2['KitchenScore'] = df2['KitchenAbvGr'] * df2['KitchenQual']
df2['FireplaceScore'] = df2['Fireplaces'] * df2['FireplaceQu']
df2['GarageScore'] = df2['GarageArea'] * df2['GarageQual']
df2['PoolScore'] = df2['PoolArea'] * df2['PoolQC']
df2['TotalBath'] = df2['BsmtFullBath'] + (0.5 * df2['BsmtHalfBath']) + df2['FullBath'] + (0.5 * df2['HalfBath'])
df2['AllSF'] = df2['GrLivArea'] + df2['TotalBsmtSF']
df2['AllFlrsSF'] = df2['1stFlrSF'] + df2['2ndFlrSF']
df2['AllPorchSF'] = df2['OpenPorchSF'] + df2['EnclosedPorch'] + df2['3SsnPorch'] + df2['ScreenPorch']
df2['HasMasVnr'] = df2.MasVnrType.replace({'BrkCmn' : 1, 'BrkFace' : 1, 'CBlock' : 1,
                                                   'Stone' : 1, 'None' : 0})
df2['SaleCondition_PriceDown'] = df2.SaleCondition.replace({'Abnorml': 1,
                                                              'Alloca': 1,
                                                              'AdjLand': 1,
                                                              'Family': 1,
                                                              'Normal': 0,
                                                              'Partial': 0})

df2['BoughtOffPlan'] = df2.SaleCondition.replace({'Abnorml' : 0, 'Alloca' : 0, 'AdjLand' : 0,
                                                          'Family' : 0, 'Normal' : 0, 'Partial' : 1})

# taken from https://www.kaggle.com/yadavsarthak/house-prices-advanced-regression-techniques/you-got-this-feature-engineering-and-lasso
df2['1stFlr_2ndFlr_Sf'] = np.log1p(df2['1stFlrSF'] + df2['2ndFlrSF'])
df2['All_Liv_SF'] = np.log1p(df2['1stFlr_2ndFlr_Sf'] + df2['LowQualFinSF'] + df2['GrLivArea'])

print(df2.shape)

In [ ]:
# check and replace any remaining missing values
df2_nan = df2.isnull().sum() / len(df2) * 100
df2_nan = df2_nan.drop(df2_nan[df2_nan == 0].index).sort_values(ascending=False)
print(df2_nan[0:5])

#for f in ls:
#    df2[f] = df2[f].apply(lambda x: x.fillna(x.median(), axis=0) # for numerical features only

In [ ]:
# transform some numerical variables to categorical
ls =['MSSubClass', 'YrSold', 'MoSold']
for f in ls:
    df2[f] = df2[f].astype(str)
    
# label encoding for categorical variables
from sklearn.preprocessing import LabelEncoder
for f in ls:
    lbl = LabelEncoder()
    lbl.fit(list(df2[f].values))
    df2[f] = lbl.transform(list(df2[f].values))
print(df2.shape)

In [ ]:
# split between numerical and categorical features
df_num = df2.select_dtypes(include = ['float64', 'int64']) # 109 features + SalePrice
num_skewed = df_num.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew': num_skewed})
print(skewness.head(5))

In [ ]:
# box-cox transformation of highly skewed features
skewness = skewness[abs(skewness) > 0.75]
skewness.drop('lat', inplace=True)
skewness.drop('lon', inplace=True)
print(skewness.shape[0])
lam=0.15
from scipy.special import boxcox1p
for f in skewness.index:
    if (f != 'lon') | (str(f)!= 'lat'):
            print(f)
            df2[f] = boxcox1p(df2[f], lam)

In [ ]:
# create dummies for categorical variables
df3 = df2.copy() #keep original df
df3 = pd.get_dummies(df3)
print(df3.shape)

In [ ]:
# split between train and test after feature engineering
train = df3[:n_train]; train['Id'] = train_id.values; train.set_index('Id')
test = df3[n_train:]; test['Id'] = test_id.values; test.set_index('Id')
outcomes = pd.DataFrame({'SalePrice': y})
outcomes['Id'] = train_id.values; outcomes.set_index('Id')

train.to_csv('train_engineered.csv')
test.to_csv('test_engineered.csv')
outcomes.to_csv('outcomes.csv')

## **FEATURES SHORTLIST**

In [ ]:
# make shortlist of features highly correlated to target variable
df4 = train.copy(); df4['SalePrice'] = y
df_xycorrs = df4.corr().iloc[:-1,-1]
features_rank = df_xycorrs[abs(df_xycorrs) > 0.3].sort_values(ascending=False)
features_shortlist = features_rank.index.tolist()
print(features_shortlist)

In [ ]:
# plot correlations between numerical features and target variable
features_shortlist.append('SalePrice')
for i in range(0, len(features_shortlist), 5):
    sns.pairplot(data=df4[features_shortlist], 
                 x_vars=df4[features_shortlist][:-1].columns[i:i+5], 
                 y_vars = ['SalePrice'])

In [ ]:
# correlation heatmap of features shortlist
df_xcorrs = df4[features_shortlist].corr()
plt.figure(figsize=(12,10))

sns.heatmap(df_xcorrs[(df_xcorrs >= 0.90) | (df_xcorrs <= -0.5)],
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1, annot=True, annot_kws={'size': 8}, square=True);
plt.tight_layout()

In [ ]:
print(*sorted(features_shortlist), sep=', \n')

In [ ]:
# features after eda and data manipulation
selection_old =['1stFlrSF','AllFlrsSF','AllSF','BsmtQual','ExterGrade','ExterQual','FireplaceQu',
                             'Foundation_PConc', 'FullBath','GarageCars','GarageScore','GarageYrBlt','KitchenQual',
                             'MasVnrArea','OverallGrade','OverallQual','TotalBath','TotalBsmtSF','TotRmsAbvGrd',
                             'YearBuilt','YearRemodAdd']


selection = ['1stFlrSF', 'AllFlrsSF', 'AllSF', 'BsmtQual', 'ExterGrade', 'ExterQual', 'FireplaceQu', 
'Foundation_PConc', 'FullBath', 'GarageArea','GarageCars', 'KitchenQual', 'OverallGrade', 'OverallQual', 
'TotRmsAbvGrd', 'TotalBath', 'YearBuilt', 'YearRemodAdd', 'lat', 'lon']

In [ ]:
# analyze selected features
#fig, ax = plt.subplots(round(len(features_shortlist) / 3), 3, figsize=(12,12))

#for i, ax in enumerate(fig.axes):
#    if i < len(features_shortlist) - 1:
#        sns.regplot(x=features_shortlist[i], y='SalePrice', data=df4[features_shortlist], ax=ax)